task name: ProduceWordBags
task description: Implement the provided C# code as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Text;

In [ ]:
var mlContext = new MLContext();

In [ ]:
public class TextData
{
    public string Text { get; set; }
}

var samples = new List<TextData>()
{
    new TextData(){ Text = "This is an example to compute bag-of-word features." },
    new TextData(){ Text = "ML.NET's ProduceWordBags API produces bag-of-word features from input text." },
    new TextData(){ Text = "It does so by first tokenizing text/string into words/tokens then computing n-grams and their numeric values." },
    new TextData(){ Text = "Each position in the output vector corresponds to a particular n-gram." },
    new TextData(){ Text = "The value at each position corresponds to, the number of times n-gram occurred in the data (Tf), or the inverse of the number of documents contain the n-gram (Idf), or compute both and multiply together (Tf-Idf)." }
};

In [ ]:
var dataview = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var textPipeline = mlContext.Transforms.Text.ProduceWordBags("BagOfWordFeatures", "Text", ngramLength: 3, useAllLengths: false, weighting: NgramExtractingEstimator.WeightingCriteria.Tf);

In [ ]:
var textTransformer = textPipeline.Fit(dataview);
var transformedDataView = textTransformer.Transform(dataview);

In [ ]:
public class TransformedTextData : TextData
{
    public float[] BagOfWordFeatures { get; set; }
}

var predictionEngine = mlContext.Model.CreatePredictionEngine<TextData, TransformedTextData>(textTransformer);

In [ ]:
var prediction = predictionEngine.Predict(samples[0]);
Console.WriteLine("Number of Features: " + prediction.BagOfWordFeatures.Length);

In [ ]:
VBuffer<ReadOnlyMemory<char>> slotNames = default;
transformedDataView.Schema["BagOfWordFeatures"].GetSlotNames(ref slotNames);
var BagOfWordFeaturesColumn = transformedDataView.GetColumn<VBuffer<float>>(transformedDataView.Schema["BagOfWordFeatures"]);
var slots = slotNames.DenseValues().ToArray();
Console.Write("N-grams: ");
foreach (var featureRow in BagOfWordFeaturesColumn)
{
    foreach (var item in featureRow.Items())
        Console.Write($"{slots[item.Key]}  ");
    Console.WriteLine();
}

Console.Write("Features: ");
for (int i = 0; i < 10; i++)
    Console.Write($"{prediction.BagOfWordFeatures[i]:F4}  ");